In [1]:
%load_ext autoreload
%autoreload 2
from bedoner.lang.juman import Japanese

In [8]:
from spacy.strings import StringStore
from spacy.vocab import Vocab

with open("../bedore-nlp-modules/data/Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt") as f:
    vs = []
    for line in f:
        vs.append(line[:-1])

s=StringStore(vs)

v=Vocab(strings=s)
v.to_disk("data/vocab.Japanese_L-12_H-768_A-12_E-30_BPE")

In [9]:
import mojimoji

In [10]:
from spacy_pytorch_transformers.pipeline.wordpiecer import PyTT_WordPiecer 

In [5]:
from spacy_pytorch_transformers._tokenizers import SerializableBertTokenizer

In [27]:
nlp = Japanese(v)
w = PyTT_WordPiecer(v)
wp = SerializableBertTokenizer("../bedore-nlp-modules/data/Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt", do_lower_case=False, tokenize_chinese_chars=False)
w.model = wp
nlp.add_pipe(w)

In [28]:
text="EXILEのATSUSHIと中島美嘉が14日ニューヨーク入り"
doc=nlp(text)

In [47]:
def model_fn_builder(
    bert_config, num_labels, init_checkpoint, use_one_hot_embeddings, max_seq_length
):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_training = mode == tf.estimator.ModeKeys.TRAIN

        (total_loss, per_example_loss, logits, predicts) = create_model(
            bert_config,
            is_training,
            input_ids,
            input_mask,
            segment_ids,
            label_ids,
            num_labels,
            use_one_hot_embeddings,
            max_seq_length,
        )

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        # scaffold_fn = None
        if init_checkpoint:
            (
                assignment_map,
                initialized_variable_names,
            ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)

        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        output_spec = None
        if mode == tf.estimator.ModeKeys.PREDICT:
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode, predictions=predicts  # {"probabilities": probabilities},
            )
        return output_spec

    return model_fn

In [50]:
from bedore_nlp_modules.bert_modules import modeling

In [53]:
from pathlib import Path

In [54]:
__dir__ = Path()

In [56]:
bert_config = modeling.BertConfig.from_json_file(str(__dir__/"data/Japanese_L-12_H-768_A-12_E-30_BPE/bert_config.json"))

W0905 15:40:53.323098 4584203712 deprecation_wrapper.py:119] From /Users/yohei_tamura/work/bedore-ner/bedore-nlp-modules/bedore_nlp_modules/bert_modules/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [62]:
import pickle

In [64]:
with open(str(__dir__ / "data/bert_result_ene_0/label2id.pkl"), "rb") as f:
    label2id = pickle.load(f)

In [66]:
num_labels=len(label2id) + 1

In [77]:
bert_dir = str(__dir__ / "data/Japanese_L-12_H-768_A-12_E-30_BPE/")
model_dir = str(__dir__ / "data/bert_result_ene_0/")

In [68]:
init_checkpoint = str(__dir__ / "data//" / "bert_model.ckpt")

In [98]:
max_seq_length=128
predict_batch_size=8
drop_remainder=False

In [99]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=num_labels,
    init_checkpoint=init_checkpoint,
    use_one_hot_embeddings=None,
    max_seq_length=max_seq_length,
)

In [100]:
run_config = tf.estimator.RunConfig(model_dir=model_dir)

In [101]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn, config=run_config, params={"batch_size": 10}
)

In [102]:
features = {}

In [103]:
def pad(a, l, c= 0):
    if len(a) >= l:
        return a[:l]
    return a + [c]*(l - len(a))

In [104]:
docs = [doc]

In [105]:
import numpy as np

In [141]:
import itertools as it

In [138]:
features["input_ids"] = np.array([pad(doc._.pytt_word_pieces, max_seq_length) for doc in docs])

In [140]:
features["input_mask"] = np.array([pad([1 for _ in range(len(doc._.pytt_word_pieces))], max_seq_length) for doc in docs])

In [155]:
segs = []
for doc in docs:
    seg = []
    for i,s in enumerate(doc._.pytt_segments):
        seg += [i] * len(s._.pytt_word_pieces)
    segs.append(pad(seg, max_seq_length))
features["segment_ids"] = np.array(segs)

In [161]:
doc._.pytt_alignment

[[1], [2], [3, 4, 5], [6], [7], [8, 9], [10], [11], [12], [13], [14]]

In [168]:
label_ids = []

In [169]:
for doc in docs:
    label_id = [label2id["[CLS]"]]
    O,X = label2id["O"], label2id["X"]
    for a in doc._.pytt_alignment:
        label_id + [O] + [X] * (len(a)-1)
    label_ids.append(pad(label_id, max_seq_length))

In [170]:
label_ids

[[406,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [164]:
label_ids.append(label_map["[CLS]"])
for i, token in enumerate(tokens):
    label_ids.append(label_map[labels[i]])

NameError: name 'label_map' is not defined